# Final Project 
### DAV 5400 Spring 2019
__Maayan Raviv__
<br>
__Dean Pienica__
<br>
<br>
_Second Chapter- Main Data Set Transformations_ 
***

### Agenda
* The data set was published in Kaggle website https://www.kaggle.com/hhs/health-insurance (We will discussed the data set in Chapter 3). 
<br>
The raw data was published in CSV format and was upload to Github platform. 
* Performing data set transformations and will clean the data.
* Uploading the data set to the SQL server. 

In [1]:
import pandas as pd 
import numpy as np
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
%matplotlib inline

# DB Credentials
username = "usgdheyiixdekm"
password = "8adc68f3bda8e3b2da6b171d207ede8f7d1c7acc91c340e4746aac2d33f41862"
host = "ec2-184-73-210-189.compute-1.amazonaws.com"
port = 5432
database = "d8t16vh0pkq35"


pd.set_option('display.max_columns', 50) # Configure pandas to show 50 columns without shorten

# highlight column of dataframe
def highlight(s):
    return ['background-color: yellow']*len(s)
# data.style.apply(highlight, subset=['State',])

In [2]:
# Read main csv
CSV_file_source = "https://raw.githubusercontent.com/maayan3007/DAV-5400/master/states.csv"
data = pd.read_csv(CSV_file_source)
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
0,Alabama,14.60%,10.10%,-4.50%,215000,2545000,165534,152206,$310,False,799176.0,910775,111599.0,989855
1,Alaska,19.90%,14.90%,-5%,36000,390000,17995,16205,$750,True,122334.0,166625,44291.0,88966
2,Arizona,16.90%,10.80%,-6.10%,410000,3288000,179445,124346,$230,True,1201770.0,1716198,514428.0,1175624
3,Arkansas,17.50%,9.50%,-8%,234000,1365000,63357,56843,$306,True,556851.0,920194,363343.0,606146
4,California,18.50%,8.60%,-9.90%,3826000,19552000,1415428,1239893,$309,True,7755381.0,11843081,4087700.0,5829777


### Data Set Clean Up
We performed the following operations:
* Null values check
* Filling Null values with mean value in each column. 
* Remove "%" from the columns. 
* Remove $ from the columns. 
* Convert data into float where necessary. 
* Remove spaces in the values and convert to string. 
* Remove "United States" record since it's a sum record of all records. 
* Create abbreviation for each state in a data dictionary. 

In [3]:
data.isnull().values.any() # Check for null cells

True

In [4]:
# Fill the null values
data = data.fillna(data.mean()) # fill null values from mean of each column

In [5]:
data.isnull().values.any() # Check for null cells
# False means there is no any cell that contains null. Every cell has a value.

True

In [6]:
# Remove %
data['Uninsured Rate (2010)'] = data['Uninsured Rate (2010)'].apply(lambda x: x.replace('%', '')) # Replace % by empty in each cell
data['Uninsured Rate (2015)'] = data['Uninsured Rate (2015)'].apply(lambda x: x.replace('%', '')) # Replace % by empty in each cell
data['Uninsured Rate Change (2010-2015)'] = data['Uninsured Rate Change (2010-2015)'].apply(lambda x: x.replace('%', '')) # Replace % by empty in each cell
data = data.astype({
    'Uninsured Rate (2010)': float, 
    'Uninsured Rate (2015)': float, 
    'Uninsured Rate Change (2010-2015)': float})
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,$310,False,799176.0,910775,111599.0,989855
1,Alaska,19.9,14.9,-5.0,36000,390000,17995,16205,$750,True,122334.0,166625,44291.0,88966
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,$230,True,1201770.0,1716198,514428.0,1175624
3,Arkansas,17.5,9.5,-8.0,234000,1365000,63357,56843,$306,True,556851.0,920194,363343.0,606146
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,$309,True,7755381.0,11843081,4087700.0,5829777


In [7]:
# Remove $
data['Average Monthly Tax Credit (2016)'] = data['Average Monthly Tax Credit (2016)'].apply(lambda x: x.replace('$', '')) # Replace $ by empty in each cell
data = data.astype({'Average Monthly Tax Credit (2016)': int})
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,310,False,799176.0,910775,111599.0,989855
1,Alaska,19.9,14.9,-5.0,36000,390000,17995,16205,750,True,122334.0,166625,44291.0,88966
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,230,True,1201770.0,1716198,514428.0,1175624
3,Arkansas,17.5,9.5,-8.0,234000,1365000,63357,56843,306,True,556851.0,920194,363343.0,606146
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,309,True,7755381.0,11843081,4087700.0,5829777


In [8]:
# Let's check the datatypes of each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 14 columns):
State                                           52 non-null object
Uninsured Rate (2010)                           52 non-null float64
Uninsured Rate (2015)                           52 non-null float64
Uninsured Rate Change (2010-2015)               52 non-null float64
Health Insurance Coverage Change (2010-2015)    52 non-null int64
Employer Health Insurance Coverage (2015)       52 non-null int64
Marketplace Health Insurance Coverage (2016)    52 non-null int64
Marketplace Tax Credits (2016)                  52 non-null int64
Average Monthly Tax Credit (2016)               52 non-null int32
State Medicaid Expansion (2013)                 51 non-null object
Medicaid Enrollment (2013)                      52 non-null float64
Medicaid Enrollment (2016)                      52 non-null int64
Medicaid Enrollment Change (2013-2016)          52 non-null float64
Medicare Enrollment (2016) 

In [9]:
# Cast Medicaid Enrollment (2013), Medicaid Enrollment (2016), and Medicaid Enrollment Change (2013-2016) as float
data = data.astype({
    'Medicaid Enrollment (2013)': float, 
    'Medicaid Enrollment (2016)': float, 
    'Medicaid Enrollment Change (2013-2016)': float})
data.info() # Check the datatypes again

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 14 columns):
State                                           52 non-null object
Uninsured Rate (2010)                           52 non-null float64
Uninsured Rate (2015)                           52 non-null float64
Uninsured Rate Change (2010-2015)               52 non-null float64
Health Insurance Coverage Change (2010-2015)    52 non-null int64
Employer Health Insurance Coverage (2015)       52 non-null int64
Marketplace Health Insurance Coverage (2016)    52 non-null int64
Marketplace Tax Credits (2016)                  52 non-null int64
Average Monthly Tax Credit (2016)               52 non-null int32
State Medicaid Expansion (2013)                 51 non-null object
Medicaid Enrollment (2013)                      52 non-null float64
Medicaid Enrollment (2016)                      52 non-null float64
Medicaid Enrollment Change (2013-2016)          52 non-null float64
Medicare Enrollment (2016

In [10]:
# Remove spaces in state column and cast to string
data['State'] = data['State'].apply(lambda x: x.strip().replace(' ', ''))
data = data.astype({'State': str})

In [11]:
# Remove UnitedStates record
data = data.drop(51)
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016)
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,310,False,799176.0,910775.0,111599.0,989855
1,Alaska,19.9,14.9,-5.0,36000,390000,17995,16205,750,True,122334.0,166625.0,44291.0,88966
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,230,True,1201770.0,1716198.0,514428.0,1175624
3,Arkansas,17.5,9.5,-8.0,234000,1365000,63357,56843,306,True,556851.0,920194.0,363343.0,606146
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,309,True,7755381.0,11843081.0,4087700.0,5829777


In [12]:
# create a new column to hold yes/no based on the state of Medicaid Expansion (2013)
def yes_no_handler(med_exp_info):
    if med_exp_info:
        return 'Yes'
    else:
        return 'No'
    
data['Medicaid Expansion'] = data['State Medicaid Expansion (2013)'].apply(lambda x: yes_no_handler(x))
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016),Medicaid Expansion
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,310,False,799176.0,910775.0,111599.0,989855,No
1,Alaska,19.9,14.9,-5.0,36000,390000,17995,16205,750,True,122334.0,166625.0,44291.0,88966,Yes
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,230,True,1201770.0,1716198.0,514428.0,1175624,Yes
3,Arkansas,17.5,9.5,-8.0,234000,1365000,63357,56843,306,True,556851.0,920194.0,363343.0,606146,Yes
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,309,True,7755381.0,11843081.0,4087700.0,5829777,Yes


In [13]:
# Create abb - state name mapping
mapping = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'DistrictofColumbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'NewHampshire': 'NH',
    'NewJersey': 'NJ',
    'NewMexico': 'NM',
    'NewYork': 'NY',
    'NorthCarolina': 'NC',
    'NorthDakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'RhodeIsland': 'RI',
    'SouthCarolina': 'SC',
    'SouthDakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'WestVirginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [14]:
data['Abbr'] = data['State'].apply(lambda x: mapping[x]) # Add above declared short names of states in to the dataframe
data.head()

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016),Medicaid Expansion,Abbr
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,310,False,799176.0,910775.0,111599.0,989855,No,AL
1,Alaska,19.9,14.9,-5.0,36000,390000,17995,16205,750,True,122334.0,166625.0,44291.0,88966,Yes,AK
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,230,True,1201770.0,1716198.0,514428.0,1175624,Yes,AZ
3,Arkansas,17.5,9.5,-8.0,234000,1365000,63357,56843,306,True,556851.0,920194.0,363343.0,606146,Yes,AR
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,309,True,7755381.0,11843081.0,4087700.0,5829777,Yes,CA


### Final data frame
The output bellow is the finalized data set after cleaning process of main csv data.
<br>
The columns in yellow are the new columns that were added to the data set. 

In [15]:
data.style.apply(highlight, subset=['Medicaid Expansion', 'Abbr',]) # Display newly added columns in highlighted

,State,Uninsured Rate (2010),Uninsured Rate (2015),Uninsured Rate Change (2010-2015),Health Insurance Coverage Change (2010-2015),Employer Health Insurance Coverage (2015),Marketplace Health Insurance Coverage (2016),Marketplace Tax Credits (2016),Average Monthly Tax Credit (2016),State Medicaid Expansion (2013),Medicaid Enrollment (2013),Medicaid Enrollment (2016),Medicaid Enrollment Change (2013-2016),Medicare Enrollment (2016),Medicaid Expansion,Abbr
0,Alabama,14.6,10.1,-4.5,215000,2545000,165534,152206,310,False,799176,910775,111599,989855,No,AL
1,Alaska,19.9,14.9,-5,36000,390000,17995,16205,750,True,122334,166625,44291,88966,Yes,AK
2,Arizona,16.9,10.8,-6.1,410000,3288000,179445,124346,230,True,1.20177e+06,1.7162e+06,514428,1175624,Yes,AZ
3,Arkansas,17.5,9.5,-8,234000,1365000,63357,56843,306,True,556851,920194,363343,606146,Yes,AR
4,California,18.5,8.6,-9.9,3826000,19552000,1415428,1239893,309,True,7.75538e+06,1.18431e+07,4.0877e+06,5829777,Yes,CA
5,Colorado,15.9,8.1,-7.8,419000,2949000,108311,67062,318,True,783420,1.37526e+06,591844,820234,Yes,CO
6,Connecticut,9.1,6,-3.1,110000,2148000,102917,80759,357,True,2.2557e+06,761137,644246,644136,Yes,CT
7,Delaware,9.7,5.9,-3.8,35000,564000,25379,21467,330,True,223324,236702,13378,186835,Yes,DE
8,DistrictofColumbia,7.6,3.8,-3.8,25000,388000,17666,1224,183,True,235786,255491,19705,90492,Yes,DC
9,Florida,21.3,13.3,-8,1597000,8847000,1531714,1428712,305,False,3.105e+06,3.64467e+06,539677,4149593,No,FL


In this step we removed the brackets from column name since when we upload that to the server it will gives an error when we call data.to_sql command.

In [16]:
# Remove '()' char from column names for the purpose of uploading records to the postgres database
data.columns = [
    'State', 
    'Uninsured Rate 2010', 
    'Uninsured Rate 2015',
    'Uninsured Rate Change 2010-2015',
    'Health Insurance Coverage Change 2010-2015',
    'Employer Health Insurance Coverage 2015',
    'Marketplace Health Insurance Coverage 2016',
    'Marketplace Tax Credits 2016', 
    'Average Monthly Tax Credit 2016',
    'State Medicaid Expansion 2013', 
    'Medicaid Enrollment 2013',
    'Medicaid Enrollment 2016', 
    'Medicaid Enrollment Change 2013-2016',
    'Medicare Enrollment 2016', 
    'Medicaid Expansion', 
    'Abbr'
]

 ### Uploading the data set to the SQL server
 DB Credentials are defined in the beginning of the jupyter script¶.
 Bellow we created the main table in the database that will hold all records of all states. 

In [17]:
engine = create_engine('postgresql://%s:%s@%s:%d/%s'%(username, password, host, port, database)) # Connect to DB
connection = engine.connect() # Get the connection object.

# Create the main table in the database in order to hold the records of states.
connection.execute('''
    CREATE TABLE IF NOT EXISTS main (
        "State" character varying(25) NOT NULL,
        "Uninsured Rate 2010" real,
        "Uninsured Rate 2015" real,
        "Uninsured Rate Change 2010-2015" real,
        "Health Insurance Coverage Change 2010-2015" integer,
        "Employer Health Insurance Coverage 2015" integer,
        "Marketplace Health Insurance Coverage 2016" integer,
        "Marketplace Tax Credits 2016" integer,
        "Average Monthly Tax Credit 2016" integer,
        "State Medicaid Expansion 2013" boolean,
        "Medicaid Enrollment 2013" real,
        "Medicaid Enrollment 2016" real,
        "Medicaid Enrollment Change 2013-2016" real,
        "Medicare Enrollment 2016" integer,
        "Medicaid Expansion" character varying(5),
        "Abbr" character varying(5)
    );
''')

In [18]:
# Update the main table from the records in the dataframe
data.to_sql("main", connection, if_exists="replace", index=False)
connection.close()